# Meeting Recorder Demo

This notebook demonstrates how to use the `meeting_recorder.py` tool for recording meeting audio from both system audio and microphone sources. We'll explore:

1. How to detect audio sources
2. How to start and stop recordings
3. How to process the recorded audio files
4. How to work with metadata files
5. How to visualize audio waveforms

## Prerequisites

Before running this notebook, make sure you have installed the required dependencies:

```bash
sudo apt install ffmpeg pulseaudio-utils
pip install librosa matplotlib numpy ipywidgets
```

## 1. Import Required Libraries

Let's start by importing the necessary libraries for our demonstration:

In [2]:
import os
import sys
import time
import json
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import IPython.display as ipd

# Import specific functions from our modules
sys.path.append('/home/rxon/projects/jenmin')  # Add project directory to path

try:
    # Import our modules
    from audio_sources import list_audio_sources, find_system_audio_source, find_microphone_source
    from rec_utils import check_dependencies, post_process_audio, get_file_duration
    
    # Check if dependencies are installed
    if not check_dependencies():
        print("Please install the required dependencies first")
except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Make sure you're running this notebook from the project directory")

## 2. Audio Source Detection

Before we start recording, we need to identify available audio sources. The `meeting_recorder.py` tool can automatically detect system audio outputs and microphone inputs using PulseAudio utilities.

Let's use functions from our `audio_sources.py` module to detect and list available audio sources:

In [6]:
# Function to get a clean list of audio sources
def get_available_audio_sources():
    """Get dictionaries of available system and microphone sources"""
    try:
        # Run the command to get audio sources
        result = subprocess.run(["pactl", "list", "sources"], 
                              capture_output=True, text=True)
        
        if result.returncode != 0:
            print("Error getting audio sources")
            return [], []
        
        sources_info = result.stdout
        current_source = None
        monitor_sources = []
        mic_sources = []
        
        # Parse the output to find sources
        for line in sources_info.split('\n'):
            if line.startswith('Source #'):
                if current_source and 'name' in current_source:
                    # Here's the key change: check the source name pattern
                    # alsa_output.* are system outputs (when monitor is True)
                    # alsa_input.* are microphone inputs
                    name = current_source.get('name', '')
                    if 'alsa_input' in name:
                        current_source['is_monitor'] = False
                        mic_sources.append(current_source)
                    elif 'monitor' in name.lower() or 'alsa_output' in name:
                        current_source['is_monitor'] = True
                        monitor_sources.append(current_source)
                    # Add fallback logic
                    elif current_source.get('is_monitor'):
                        monitor_sources.append(current_source)
                    else:
                        mic_sources.append(current_source)
                
                current_source = {'id': line.split('#')[1].strip()}
            elif current_source is not None:
                if 'Name: ' in line:
                    current_source['name'] = line.split('Name: ')[1].strip()
                    # Pre-identify based on name pattern
                    name = current_source['name']
                    if 'alsa_input' in name:
                        current_source['is_monitor'] = False
                    elif 'monitor' in name.lower() or 'alsa_output' in name:
                        current_source['is_monitor'] = True
                elif 'monitor' in line.lower() and 'alsa_output' in current_source.get('name', ''):
                    # Confirm monitor status for output devices
                    current_source['is_monitor'] = True
                elif 'Description:' in line and 'alsa_input' in current_source.get('name', ''):
                    # Confirm microphone status for input devices
                    current_source['is_monitor'] = False
                elif 'State: ' in line:
                    current_source['state'] = line.split('State: ')[1].strip()
        
        # Add the last source
        if current_source and 'name' in current_source:
            name = current_source.get('name', '')
            if 'alsa_input' in name:
                current_source['is_monitor'] = False
                mic_sources.append(current_source)
            elif 'monitor' in name.lower() or 'alsa_output' in name:
                current_source['is_monitor'] = True
                monitor_sources.append(current_source)
            elif current_source.get('is_monitor'):
                monitor_sources.append(current_source)
            else:
                mic_sources.append(current_source)
        
        return monitor_sources, mic_sources
    except Exception as e:
        print(f"Error getting audio sources: {e}")
        return [], []

# Get and display audio sources
monitor_sources, mic_sources = get_available_audio_sources()

print("=== System Audio Sources ===")
for i, source in enumerate(monitor_sources):
    print(f"{i+1}. {source['name']} ({source.get('state', 'unknown')})")

print("\n=== Microphone Sources ===")
for i, source in enumerate(mic_sources):
    print(f"{i+1}. {source['name']} ({source.get('state', 'unknown')})")

# Find default sources
system_source = find_system_audio_source() if 'find_system_audio_source' in globals() else None
mic_source = find_microphone_source() if 'find_microphone_source' in globals() else None

if system_source:
    print(f"\nRecommended system audio source: {system_source}")
if mic_source:
    print(f"Recommended microphone source: {mic_source}")

=== System Audio Sources ===
1. alsa_output.pci-0000_06_00.6.analog-stereo.monitor (SUSPENDED)
2. alsa_output.usb-0b0e_Jabra_Link_370_745C4BE64CEC-00.iec958-stereo.monitor (SUSPENDED)

=== Microphone Sources ===
1. alsa_input.pci-0000_06_00.6.analog-stereo (SUSPENDED)
2. alsa_input.usb-0b0e_Jabra_Link_370_745C4BE64CEC-00.mono-fallback (SUSPENDED)
3. alsa_input.usb-Angetube_Live_Camera_Angetube_Live_Camera_20211101015-02.analog-stereo (SUSPENDED)

Recommended system audio source: alsa_output.pci-0000_06_00.6.analog-stereo.monitor


## 3. Recording Functions

Now we'll create functions to handle recording from the selected audio sources using ffmpeg. This is similar to the functionality in the `meeting_recorder.py` tool, but simplified for demonstration purposes.

In [4]:
class SimpleRecorder:
    def __init__(self, output_dir="~/Recordings/Meetings", format="mp3", bitrate="192k",
                 source_system=None, source_mic=None, combined=True):
        # Initialize configuration
        self.output_dir = os.path.expanduser(output_dir)
        self.format = format
        self.bitrate = bitrate
        self.system_source = source_system
        self.mic_source = source_mic
        self.combined = combined
        
        # Initialize state variables
        self.ffmpeg_process = None
        self.recording = False
        self.recording_started = None
        self.current_output_path = None
        
        # Setup
        os.makedirs(self.output_dir, exist_ok=True)
        
    def get_audio_sources_args(self):
        """Get ffmpeg arguments for audio sources based on configuration"""
        if self.combined and self.system_source and self.mic_source:
            # Return combined recording setup
            return [
                "-f", "pulse", "-i", self.system_source,
                "-f", "pulse", "-i", self.mic_source,
                "-filter_complex", "amix=inputs=2:duration=longest"
            ]
        elif self.system_source:
            # Return system audio only
            return ["-f", "pulse", "-i", self.system_source]
        elif self.mic_source:
            # Return microphone only
            return ["-f", "pulse", "-i", self.mic_source]
        else:
            # Default fallback
            return ["-f", "pulse", "-i", "default"]
    
    def start_recording(self, name=None):
        """Start recording with optional custom name"""
        if self.ffmpeg_process:
            self.stop_recording()
            
        # Create output directory based on date
        date_folder = datetime.now().strftime("%Y-%m-%d")
        dir_path = os.path.join(self.output_dir, date_folder)
        os.makedirs(dir_path, exist_ok=True)
        
        # Create filename with timestamp
        timestamp = datetime.now().strftime("%H%M%S")
        if name:
            # Sanitize name
            for char in ['/', '\\', ':', '*', '?', '"', '<', '>', '|']:
                name = name.replace(char, '_')
            filename = f"{name}_{timestamp}.{self.format}"
        else:
            filename = f"recording_{timestamp}.{self.format}"
        
        output_path = os.path.join(dir_path, filename)
        
        # Get audio input arguments based on selected sources
        input_args = self.get_audio_sources_args()
        
        try:
            # Build ffmpeg command
            cmd = [
                "ffmpeg", "-v", "warning", "-stats", 
                *input_args,
                "-c:a", "libmp3lame" if self.format == "mp3" else "pcm_s16le",
                "-b:a", self.bitrate, "-y", output_path
            ]
            
            print(f"Starting recording: {output_path}")
            
            # Start recording process
            self.ffmpeg_process = subprocess.Popen(
                cmd, 
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            
            # Check if process started correctly
            time.sleep(1)
            if self.ffmpeg_process.poll() is not None:
                print(f"Error: ffmpeg failed to start (exit code {self.ffmpeg_process.returncode})")
                return None
            
            self.recording = True
            self.recording_started = datetime.now()
            self.current_output_path = output_path
            
            # Save initial metadata
            metadata = {
                "start_time": self.recording_started.isoformat(),
                "sources": {
                    "system": self.system_source,
                    "mic": self.mic_source,
                    "combined": self.combined
                },
                "format": self.format,
                "bitrate": self.bitrate
            }
            
            # Write metadata to file
            metadata_path = f"{os.path.splitext(output_path)[0]}.json"
            with open(metadata_path, 'w') as f:
                json.dump(metadata, f, indent=2)
            
            return output_path
        except Exception as e:
            print(f"Error starting recording: {e}")
            return None
    
    def stop_recording(self):
        """Stop the current recording"""
        if not self.ffmpeg_process:
            return None
            
        now = datetime.now()
        print(f"Stopping recording...")
        
        # Store output path
        output_path = self.current_output_path
        
        # Calculate duration
        duration = None
        if self.recording_started:
            duration = (now - self.recording_started).total_seconds()
        
        # Terminate ffmpeg process
        self.ffmpeg_process.terminate()
        try:
            self.ffmpeg_process.wait(timeout=5)
        except subprocess.TimeoutExpired:
            print("Warning: ffmpeg process didn't exit, forcing termination")
            self.ffmpeg_process.kill()
        
        self.ffmpeg_process = None
        self.recording = False
        
        print(f"Recording stopped")
        
        # Update metadata
        if output_path and os.path.exists(output_path):
            # Update metadata with final information
            metadata_path = f"{os.path.splitext(output_path)[0]}.json"
            
            try:
                # Read existing metadata if available
                if os.path.exists(metadata_path):
                    with open(metadata_path, 'r') as f:
                        metadata = json.load(f)
                else:
                    metadata = {}
                
                # Get accurate duration from the file
                file_duration = get_file_duration(output_path)
                if not file_duration and duration:
                    file_duration = duration
                
                # Update metadata
                metadata.update({
                    "end_time": now.isoformat(),
                    "duration_seconds": file_duration,
                    "file_size_mb": os.path.getsize(output_path) / (1024 * 1024)
                })
                
                # Write updated metadata
                with open(metadata_path, 'w') as f:
                    json.dump(metadata, f, indent=2)
                    
            except Exception as e:
                print(f"Error updating metadata: {e}")
            
        self.recording_started = None
        self.current_output_path = None
        
        return output_path

## 4. File Organization and Naming

The meeting recorder organizes files by date and timestamps. Let's explore the directory structure and file naming conventions:

In [5]:
# Create a demo recording with our recorder
def create_demo_recording(duration=5):
    """Create a short demo recording"""
    # Create recorder with default sources
    recorder = SimpleRecorder(
        output_dir="~/Recordings/Demo",
        source_system=system_source,
        source_mic=mic_source,
        combined=True if system_source and mic_source else False
    )
    
    # Show selected sources
    print(f"Recording with system source: {recorder.system_source}")
    print(f"Recording with mic source: {recorder.mic_source}")
    print(f"Combined mode: {recorder.combined}")
    
    # Start recording
    output_path = recorder.start_recording(name="demo_recording")
    
    if not output_path:
        print("Failed to start recording")
        return None
    
    print(f"Recording for {duration} seconds...")
    time.sleep(duration)
    
    # Stop recording
    output_path = recorder.stop_recording()
    
    # Print recording details
    if output_path and os.path.exists(output_path):
        print("\nRecording completed:")
        print(f"- File: {os.path.basename(output_path)}")
        print(f"- Path: {output_path}")
        print(f"- Size: {os.path.getsize(output_path) / 1024:.1f} KB")
        
        # Print directory structure
        base_dir = os.path.dirname(os.path.dirname(output_path))
        print("\nDirectory structure:")
        
        for root, dirs, files in os.walk(base_dir):
            level = root.replace(base_dir, '').count(os.sep)
            indent = ' ' * 4 * level
            print(f"{indent}{os.path.basename(root)}/")
            sub_indent = ' ' * 4 * (level + 1)
            for f in files:
                print(f"{sub_indent}{f}")
                
        return output_path
    else:
        print("No recording output available")
        return None

# Create a short demo recording (uncomment to run)
# demo_recording_path = create_demo_recording(3)

# Example directory structure and file naming (without actually recording)
demo_date = datetime.now().strftime("%Y-%m-%d")
demo_time = datetime.now().strftime("%H%M%S")

print("Example directory structure and naming convention:")
print(f"~/Recordings/Meetings/")
print(f"└── {demo_date}/")
print(f"    ├── recording_{demo_time}.mp3")
print(f"    └── recording_{demo_time}.json")
print(f"    ├── meeting_standup_{demo_time}.mp3")
print(f"    └── meeting_standup_{demo_time}.json")
print("\nCustom naming examples:")
print("- meeting_standup_103045.mp3")
print("- team_interview_153022.mp3")
print("- project_planning_093015.mp3")

Example directory structure and naming convention:
~/Recordings/Meetings/
└── 2025-07-28/
    ├── recording_171313.mp3
    └── recording_171313.json
    ├── meeting_standup_171313.mp3
    └── meeting_standup_171313.json

Custom naming examples:
- meeting_standup_103045.mp3
- team_interview_153022.mp3
- project_planning_093015.mp3


## 5. Audio Post-Processing

One of the key features of our meeting recorder is the ability to apply post-processing effects to improve audio quality. Let's explore the post-processing capabilities, including:

1. Noise reduction
2. Volume normalization
3. Speech enhancement

We'll use our `post_process_audio` function from `rec_utils.py` and visualize the audio waveforms before and after processing.

In [ ]:
# First, let's create a function to visualize audio waveforms
def visualize_audio_waveform(file_path, title=None):
    """Visualize the waveform of an audio file"""
    try:
        # Use ffmpeg to convert audio to raw PCM for processing
        import tempfile
        import wave
        
        # Create temporary wav file
        with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp:
            temp_wav = tmp.name
        
        # Convert audio to WAV using ffmpeg
        cmd = ["ffmpeg", "-y", "-i", file_path, "-acodec", "pcm_s16le", temp_wav]
        subprocess.run(cmd, check=True, capture_output=True)
        
        # Read WAV file
        with wave.open(temp_wav, 'rb') as wf:
            # Get basic info
            n_channels = wf.getnchannels()
            sample_width = wf.getsampwidth()
            framerate = wf.getframerate()
            n_frames = wf.getnframes()
            
            # Read frames
            buffer = wf.readframes(n_frames)
            
            # Convert buffer to numpy array
            dtype = {1: np.int8, 2: np.int16, 4: np.int32}[sample_width]
            audio_data = np.frombuffer(buffer, dtype=dtype)
            
            # For stereo, average the channels
            if n_channels == 2:
                audio_data = audio_data.reshape(-1, 2)
                audio_data = audio_data.mean(axis=1)
            
            # Normalize
            audio_data = audio_data / np.max(np.abs(audio_data))
            
            # Generate time axis
            duration = n_frames / framerate
            time = np.linspace(0, duration, len(audio_data))
            
            # Plot the waveform
            plt.figure(figsize=(12, 4))
            plt.plot(time, audio_data)
            plt.xlabel('Time (s)')
            plt.ylabel('Amplitude')
            plt.title(title or f"Audio Waveform: {os.path.basename(file_path)}")
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        
        # Clean up
        os.unlink(temp_wav)
        
        return {
            'n_channels': n_channels,
            'framerate': framerate,
            'duration': duration,
        }
    except Exception as e:
        print(f"Error visualizing audio: {e}")
        return None

# Now, let's define a function to apply post-processing and visualize results
def demo_post_processing(input_file):
    """Apply post-processing to a recording and visualize before/after waveforms"""
    if not os.path.exists(input_file):
        print(f"File not found: {input_file}")
        return
    
    print(f"Processing file: {input_file}")
    
    # Create output filenames for each type of processing
    base_path = os.path.splitext(input_file)[0]
    ext = os.path.splitext(input_file)[1]
    noise_reduced_file = f"{base_path}_noise_reduced{ext}"
    normalized_file = f"{base_path}_normalized{ext}"
    enhanced_file = f"{base_path}_enhanced{ext}"
    
    # First, visualize the original waveform
    print("\nOriginal audio:")
    visualize_audio_waveform(input_file, "Original Audio")
    
    # Apply noise reduction
    print("\nApplying noise reduction...")
    post_process_audio(input_file, noise_reduced_file, noise_reduce=True)
    if os.path.exists(noise_reduced_file):
        visualize_audio_waveform(noise_reduced_file, "Noise Reduced Audio")
    
    # Apply normalization
    print("\nApplying volume normalization...")
    post_process_audio(input_file, normalized_file, normalize=True)
    if os.path.exists(normalized_file):
        visualize_audio_waveform(normalized_file, "Normalized Audio")
    
    # Apply speech enhancement
    print("\nApplying speech enhancement...")
    post_process_audio(input_file, enhanced_file, enhance_speech=True)
    if os.path.exists(enhanced_file):
        visualize_audio_waveform(enhanced_file, "Speech Enhanced Audio")
    
    # Apply all effects together
    all_effects_file = f"{base_path}_all_effects{ext}"
    print("\nApplying all effects...")
    post_process_audio(input_file, all_effects_file, 
                      noise_reduce=True, normalize=True, enhance_speech=True)
    if os.path.exists(all_effects_file):
        visualize_audio_waveform(all_effects_file, "All Effects Applied")
    
    print("\nPost-processing complete. Files created:")
    print(f"- Noise reduced: {os.path.basename(noise_reduced_file)}")
    print(f"- Normalized: {os.path.basename(normalized_file)}")
    print(f"- Speech enhanced: {os.path.basename(enhanced_file)}")
    print(f"- All effects: {os.path.basename(all_effects_file)}")

# To run post-processing on a demo file, first create a recording, then:
# demo_post_processing(demo_recording_path)

# Alternatively, we can use a sample audio file if available
sample_file = "/home/rxon/projects/jenmin/sample_recording.mp3"
if os.path.exists(sample_file):
    print(f"Sample file found: {sample_file}")
    # Uncomment to run post-processing
    # demo_post_processing(sample_file)
else:
    print("To demo post-processing, either:")
    print("1. Create a recording using create_demo_recording() function")
    print("2. Create a sample_recording.mp3 file in the project directory")

## 6. Command Line Interface

The `meeting_recorder.py` tool has a comprehensive command-line interface. Let's explore the available options and how to use them:

In [ ]:
# Let's create a function to display the command line options
def show_cli_options():
    """Show the command line options for meeting_recorder.py"""
    print("Command Line Options for meeting_recorder.py:")
    print("\n= Basic Usage =")
    print("python meeting_recorder.py                 # Start in interactive mode")
    print("python meeting_recorder.py --start         # Start recording immediately")
    print("python meeting_recorder.py --list-sources  # List available audio sources")
    
    print("\n= Audio Source Options =")
    print("--source-system SOURCE   # Specify system audio source")
    print("--source-mic SOURCE      # Specify microphone source")
    print("--system-only            # Record only system audio (no microphone)")
    print("--mic-only               # Record only microphone (no system audio)")
    
    print("\n= Output Options =")
    print("--output-dir DIR         # Output directory for recordings (default: ~/Recordings/Meetings)")
    print("--format FORMAT          # Audio format: mp3 or wav (default: mp3)")
    print("--bitrate BITRATE        # Bitrate for audio encoding (default: 192k)")
    print("--name NAME              # Custom name prefix for recordings")
    
    print("\n= Processing Options =")
    print("--post-process           # Apply post-processing after recording stops")
    
    print("\n= Examples =")
    print("# List available sources:")
    print("python meeting_recorder.py --list-sources")
    print("\n# Start recording with custom name and sources:")
    print("python meeting_recorder.py --start --name team_meeting --source-system 'alsa_output.pci-0000_00_1f.3.analog-stereo.monitor' --source-mic 'alsa_input.pci-0000_00_1f.3.analog-stereo'")
    print("\n# Record only system audio with post-processing:")
    print("python meeting_recorder.py --start --system-only --post-process")

# Display the command line options
show_cli_options()

# We can also run subprocess to show the actual help output from the script
def show_script_help():
    """Run the script with --help to show actual help output"""
    try:
        script_path = "/home/rxon/projects/jenmin/meeting_recorder.py"
        if os.path.exists(script_path):
            result = subprocess.run(["python", script_path, "--help"], capture_output=True, text=True)
            if result.returncode == 0:
                print(result.stdout)
            else:
                print("Error running script with --help")
                print(result.stderr)
        else:
            print(f"Script not found at {script_path}")
    except Exception as e:
        print(f"Error: {e}")

# Uncomment to show the actual help output from the script
# show_script_help()

## 7. Implementation Demo

Let's put everything together and demonstrate the complete workflow of the meeting recorder:

1. Detect audio sources
2. Start recording
3. Stop recording
4. Work with metadata
5. Apply post-processing
6. Visualize results

Below is a complete demo that simulates using the meeting recorder for capturing a meeting:

In [ ]:
# Full implementation demo with widgets for interactive controls
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    # 1. Create recorder with detected sources
    demo_recorder = SimpleRecorder(
        output_dir="~/Recordings/MeetingDemo",
        source_system=system_source,
        source_mic=mic_source,
        combined=True if system_source and mic_source else False
    )
    
    # 2. Display current setup
    print("Meeting Recorder Demo Setup")
    print("==========================")
    print(f"System audio source: {demo_recorder.system_source}")
    print(f"Microphone source: {demo_recorder.mic_source}")
    print(f"Combined mode: {demo_recorder.combined}")
    print(f"Output directory: {demo_recorder.output_dir}")
    print(f"Format: {demo_recorder.format}")
    print(f"Bitrate: {demo_recorder.bitrate}")
    print("\nNote: This is a demonstration notebook. For real usage,")
    print("use the meeting_recorder.py script from the command line.")
    
    # 3. Create control widgets
    recording_name = widgets.Text(
        value='demo_meeting',
        placeholder='Enter recording name',
        description='Name:',
        disabled=False
    )
    
    status_output = widgets.Output()
    
    def start_recording_button_clicked(b):
        with status_output:
            clear_output()
            print("Starting recording...")
            output_path = demo_recorder.start_recording(recording_name.value)
            if output_path:
                print(f"Recording to: {output_path}")
                print("\nPress 'Stop Recording' to end the recording")
                b.disabled = True
                stop_button.disabled = False
            else:
                print("Failed to start recording!")
                
    def stop_recording_button_clicked(b):
        with status_output:
            clear_output()
            print("Stopping recording...")
            output_path = demo_recorder.stop_recording()
            b.disabled = True
            start_button.disabled = False
            if output_path:
                print(f"Recording saved to: {output_path}")
                
                # Read and display metadata
                metadata_path = f"{os.path.splitext(output_path)[0]}.json"
                if os.path.exists(metadata_path):
                    try:
                        with open(metadata_path, 'r') as f:
                            metadata = json.load(f)
                            print("\nRecording Metadata:")
                            print(json.dumps(metadata, indent=2))
                            
                        # Update process button to use this file
                        process_button.description = "Process Recording"
                        process_button.disabled = False
                        process_button.file_path = output_path
                    except Exception as e:
                        print(f"Error reading metadata: {e}")
                        
    def process_recording_button_clicked(b):
        if hasattr(b, 'file_path') and os.path.exists(b.file_path):
            with status_output:
                clear_output()
                print(f"Processing recording: {b.file_path}")
                demo_post_processing(b.file_path)
        else:
            with status_output:
                clear_output()
                print("No recording file selected for processing")
    
    # Create buttons
    start_button = widgets.Button(
        description='Start Recording',
        disabled=False,
        button_style='success',
        tooltip='Start a new recording',
        icon='play'
    )
    start_button.on_click(start_recording_button_clicked)
    
    stop_button = widgets.Button(
        description='Stop Recording',
        disabled=True,
        button_style='danger',
        tooltip='Stop the current recording',
        icon='stop'
    )
    stop_button.on_click(stop_recording_button_clicked)
    
    process_button = widgets.Button(
        description='No Recording',
        disabled=True,
        button_style='info',
        tooltip='Process the recording',
        icon='cogs'
    )
    process_button.on_click(process_recording_button_clicked)
    
    # Layout widgets
    controls = widgets.HBox([start_button, stop_button, process_button])
    
    # Display the interface
    display(recording_name)
    display(controls)
    display(status_output)
    
    with status_output:
        print("Ready to record. Press 'Start Recording' to begin.")
        
except ImportError:
    print("Note: ipywidgets not available. For interactive controls, install with:")
    print("pip install ipywidgets")
    print("\nAlternatively, you can use the following functions directly:")
    print("- create_demo_recording(duration)")
    print("- demo_post_processing(file_path)")
    
# For non-widget usage, you can run these functions:
# recording_path = create_demo_recording(duration=5)  # 5 seconds
# demo_post_processing(recording_path)

## Conclusion

In this notebook, we've explored the meeting recorder tool that was adapted from the original Spotirec project. The key features include:

1. **Audio Source Detection**: Automatic detection and selection of system audio and microphone sources
2. **Recording Control**: Simple start/stop controls for recording meetings
3. **File Organization**: Date-based organization structure for easy management
4. **Metadata Storage**: JSON-based metadata storage with recording details
5. **Post-Processing**: Audio enhancement features for improving recording quality

### Next Steps

To use the full meeting recorder tool:

1. Run the standalone script:
   ```bash
   python meeting_recorder.py
   ```

2. For recording with specific sources:
   ```bash
   python meeting_recorder.py --source-system <system_source> --source-mic <mic_source> --start
   ```

3. For interactive mode:
   ```bash
   python meeting_recorder.py
   ```
   Then follow the on-screen prompts to start, stop, and process recordings.

### Further Development Ideas

- Add support for automatic meeting detection
- Implement cloud storage integration for recordings
- Add speech-to-text transcription
- Create a web interface for remote control
- Add video recording capabilities